# Production Code

In [ ]:
import os
import re
import numpy as np
import pandas as pd

source_directory = "../surveys/plain_txt/"

## Extract

### Global Variables

### Helper Functions

In [ ]:
def find_between(str_1, str_2, contents):
    regex_between = str_1 + r"([\w\W]*)" + str_2
    regex_search = re.findall(regex_between, contents)

    return regex_search

### Parsing Functions

In [ ]:
def parse_interviewee(contents):
    name_regex = r"nombre de la persona[^\n:]*:[\s]*([^\n]*)\n"
    name_search = re.findall(name_regex, contents)
    if len(name_search) == 1:
        name = name_search[0]
        if name == "3":
            name = "N/A"
    else:
        alt_name_regex = r"nombre de la persona[^\n\(]*\(\w* +([^\n]*)\n"
        alt_name_search = re.findall(alt_name_regex, contents)
        if len(alt_name_search) == 1:
            name = alt_name_search[0]
            if name == "3":
                name = "N/A"
        else:
            name = "UNENCOUNTERED FORMAT"

    name = name.strip()

    return name


def parse_date_range(section_search):
    date_range = ""

    if len(section_search) == 1:
        section_string = section_search[0]

        date_regex = r"([\w ]+\d+ *[-–] *[\w ]+\d+)"
        date_search = re.findall(date_regex, section_string)

        if len(date_search) == 1:
            date_range = date_search[0]
        else:
            date_range = "UNENCOUNTERED FORMAT"
    else:
        date_range = "UNENCOUNTERED FORMAT"

    return date_range


def parse_section_6_1(section_search, column_headers):
    section_6_1_data = []

    return section_6_1_data

### Process the Whole File

In [ ]:
def process_files(source_directory):
    dataset = []

    for process_file in os.listdir(source_directory):
        file_path = os.path.join(source_directory, process_file)

        # with statements automatically control the closing of files
        with open(file_path, "r") as file:
            contents = file.read()
            contents = contents.lower()

            section_start = "v-4: otros ingresos"
            section_end = "sección vi:"
            section_search = find_between(section_start, section_end, contents)

            interviewee = parse_interviewee(contents)
            date_range = parse_date_range(section_search)
            section_6_1_data = parse_section_6_1(section_search)

            data_dict = {}

            # check to make sure files contain the right amount of entries
            if len(section_6_1_data) <= 1:
                print(f"Unencountered Format: {process_file}\n"
                      + f"Length of section_data: {len(section_6_1_data)}\n")

            for row in section_6_1_data:
                data_dict = {}
                data_dict["filename"] = process_file
                data_dict["interviewee"] = interviewee
                data_dict["date_range"] = date_range

                for key, value in row.items():
                    data_dict[key] = value

                dataset.append(data_dict)

    # convert list to DataFrame
    raw_df = pd.DataFrame(data=dataset)

    return raw_df

## Transform

### Helper Functions

### Pipeline Functions

In [ ]:
def start_pipeline(dataf):
    return dataf.copy()


### Create Raw and Cleaned DataFrame

In [ ]:
raw_df = process_files(source_directory)

clean_df = (raw_df
            .pipe(start_pipeline))

## Other
### Functions to Check the Implementation

In [ ]:
def get_dropped_df(raw_dataf, clean_dataf):
    raw_indexes = raw_dataf.index
    clean_indexes = clean_dataf.index

    dropped_indexes = raw_indexes[~raw_indexes.isin(clean_indexes)]
    dropped_df = raw_dataf.loc[dropped_indexes].copy()

    return dropped_df

### Create Dropped DataFrame

In [ ]:
dropped_df = get_dropped_df(raw_df, clean_df)

## Load
### Export to CSV

In [ ]:
datasets_directory = "../datasets/"
filename = "section_6_1.csv"
file_path = os.path.join(datasets_directory, filename)

clean_df.to_csv(file_path, index=False, na_rep="null")

### Create Dropped DataFrame

***
# Testing Code
### View the Data

In [ ]:
clean_df.head(50)

### Check for Parsing Errors

In [ ]:
cols_to_check = clean_df.columns
cols_to_drop = ["filename", "interviewee"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(clean_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Check Dropped Rows
Another way to check is by opening the variable inspector and manually scrolling
through the `df_dropped` DataFrame. 

The code below tells us whether or not there are files that contain no information for this section. If raw file count is the same as clean file count, then no files were dropped.

In [ ]:
print("Raw File Count: {}".format(len(raw_df["filename"].value_counts())))
print("Clean File Count: {}".format(len(clean_df["filename"].value_counts())))

If there are files that are completely dropped, find those files and look at them to check for parsing errors.

In [ ]:
raw_file_names = raw_df["filename"].value_counts().index
clean_file_names = clean_df["filename"].value_counts().index

raw_file_names[~raw_file_names.isin(clean_file_names)]

Check the rows value counts to see if any rows are being dropped that should not be.

In [ ]:
cols_to_check = dropped_df.columns
cols_to_drop = ["filename", "interviewee"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(dropped_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Duplicate Rows

In [ ]:
df_columns = clean_df.columns[clean_df.columns != "filename"]
clean_df[clean_df.loc[:, df_columns].duplicated(keep=False)]

## Experimental Code

In [ ]:
# -------------------------------------------------------------------------- #
# TESTING CODE
# process_file = "-22 Reinaldo Cuellar.txt"
process_file = "Encuesta N° 25 - FAO Rene Gaspar.txt"
# process_file = "Encuesta N° 2 - FAO - Mariana Quintupuray.txt"

file_path = os.path.join(source_directory, process_file)

file = open(file_path, "r")

contents = file.read()
contents = contents.lower()

section_start = ""
section_end = ""
section_search = find_between(section_start, section_end, contents)